In [1]:
import os
#execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [3]:
playHistoryPath = '/data/sample264'
playHistoryGraph = sparkSession.read.parquet(playHistoryPath)
playHistoryGraph.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- trackId: integer (nullable = true)
 |-- artistId: integer (nullable = true)
 |-- timestamp: long (nullable = true)



In [4]:
metaDataPath = '/data/meta'
metaDataGraph = sparkSession.read.parquet(metaDataPath)
metaDataGraph.printSchema()

root
 |-- type: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Id: integer (nullable = true)



In [5]:
playHistoryGraph.createTempView("history1")
playHistoryGraph.createTempView("history2")

# Task 1

In [ ]:
from pyspark.sql.functions import count, col

consecutiveTracksForUser = sparkSession.sql( \
    "select h1.trackId as track1, h2.trackId as track2, h1.userId as user " \
    "from history1 h1, history2 h2 " \
    "where h1.userId = h2.userId " \
    "and h1.trackId != h2.trackId " \
    "and abs(h2.timestamp - h1.timestamp) <= 420 " \
).groupBy(col("track1"), col("track2")) \
.count().alias("count") \
.orderBy(col("track1"), col("track2")) \
.cache()

In [ ]:
#consecutiveTracksForUser.show()

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, sum

window = Window.partitionBy("track1").orderBy(col("count").desc())
       
topsDF = consecutiveTracksForUser.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= 40) \
        .drop(col("row_number")) \
        .orderBy(col("track1"), col("track2")) \
        .cache()

In [ ]:
#topsDF.show()

In [ ]:

sumsDF = topsDF.groupBy(col("track1")) \
.agg(sum(col("count")).alias("sum_weights")) \
.orderBy("track1") \
.cache()

In [ ]:
#sumsDF.show()

In [ ]:
normalized_count = topsDF.join(sumsDF, "track1", "inner") \
    .withColumn("weight", col("count") / col("sum_weights")) \
    .cache()


In [ ]:
#normalized_count.show()

In [ ]:
results = normalized_count.orderBy(col("weight").desc(), col("track1"), col("track2")).limit(40)
#results.show()

In [ ]:
results = results.select(col("track1"), col("track2"))
for t1, t2 in results.collect():
    print("{}\t{}".format(t1,t2))

# Task 2

In [ ]:
from pyspark.sql.functions import count, col

tracksPerUser = sparkSession.sql( \
    "select userId as user, trackId as track " \
    "from history1 "
).groupBy(col("user"), col("track")) \
.count().alias("count") \
.orderBy(col("count").desc(), col("user"), col("track")) \
.cache()

In [ ]:
tracksPerUser.show()

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, sum

window = Window.partitionBy("user").orderBy(col("count").desc())
       
topsTracksPerUser = tracksPerUser.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= 1000) \
        .drop(col("row_number")) \
        .cache()

In [ ]:
sumsTopsTracksPerUser = topsTracksPerUser.groupBy(col("user")) \
.agg(sum(col("count")).alias("sum_weights")) \
.orderBy("user") \
.cache()

In [ ]:
normalized_topsTracksPerUser = topsTracksPerUser.join(sumsTopsTracksPerUser, "user", "inner") \
    .withColumn("norm_weight", col("count") / col("sum_weights")) \
    .cache()

In [ ]:
results = normalized_topsTracksPerUser.orderBy(col("norm_weight").desc(), col("user"), col("track")) \
    .limit(40) \
    .select(col("user"),col("track"))

In [ ]:
for u, t in results.collect():
    print("{} {}".format(u, t))

# Task 3

In [6]:
from pyspark.sql.functions import count, col

artistsPerUser = sparkSession.sql( \
    "select userId as user, artistId as artist " \
    "from history1 "
).groupBy(col("user"), col("artist")) \
.count().alias("count") \
.orderBy(col("count").desc(), col("user"), col("artist")) \
.cache()

In [7]:
artistsPerUser.show()

+------+-------+-----+
|  user| artist|count|
+------+-------+-----+
|668849| 994686|  277|
|436158|1003021|  142|
|442306|1001300|  107|
|560428| 975695|   94|
|767478| 991179|   94|
|278647| 981306|   87|
|770607| 978956|   76|
| 20167| 978800|   75|
| 26976|1003021|   75|
|637446| 978288|   71|
|397054| 974777|   70|
|407962| 968823|   70|
|201788| 970240|   69|
|343313| 968823|   69|
|510688| 976986|   69|
|408783| 978956|   67|
|714890| 985755|   67|
|525436| 976384|   66|
|607295| 993789|   63|
|300275| 989189|   62|
+------+-------+-----+
only showing top 20 rows



In [8]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, sum

window = Window.partitionBy("user").orderBy(col("count").desc())
       
topsArtistsPerUser = artistsPerUser.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= 100) \
        .drop(col("row_number")) \
        .cache()

In [9]:
sumsTopsArtistsPerUser = topsArtistsPerUser.groupBy(col("user")) \
.agg(sum(col("count")).alias("sum_weights")) \
.orderBy("user") \
.cache()

In [10]:
normalized_topsArtistsPerUser = topsArtistsPerUser.join(sumsTopsArtistsPerUser, "user", "inner") \
    .withColumn("norm_weight", col("count") / col("sum_weights")) \
    .cache()

In [11]:
results = normalized_topsArtistsPerUser.orderBy(col("norm_weight").desc(), col("user"), col("artist")) \
    .limit(40) \
    .select(col("user"),col("artist"))

In [12]:
for u, t in results.collect():
    print("{} {}".format(u, t))

66 993426
116 974937
128 1003021
131 983068
195 997265
215 991696
235 990642
288 1000564
300 1003362
321 986172
328 967986
333 1000416
346 982037
356 974846
374 1003167
428 993161
431 969340
445 970387
488 970525
542 969751
612 987351
617 970240
649 973851
658 973232
662 975279
698 995788
708 968848
746 972032
747 972032
776 997265
784 969853
806 995126
811 996436
837 989262
901 988199
923 977066
934 990860
957 991171
989 975339
999 968823


# Task 4

In [13]:
from pyspark.sql.functions import count, col

tracksPerArtist = sparkSession.sql( \
    "select artistId as artist, trackId as track " \
    "from history1 "
).groupBy(col("artist"), col("track")) \
.count().alias("count") \
.orderBy(col("count").desc(), col("artist"), col("track")) \
.cache()

In [14]:
tracksPerArtist.show()

+------+------+-----+
|artist| track|count|
+------+------+-----+
|987351|886091| 2958|
|988199|871513| 2904|
|997265|946408| 2836|
|981306|864690| 2582|
|974503|858904| 2453|
|981306|831005| 2308|
|970525|841340| 2275|
|983300|846624| 2101|
|977324|957460| 1909|
|991179|870292| 1714|
|981306|940362| 1521|
|969751|815388| 1504|
|990305|939606| 1464|
|970511|825174| 1441|
|969751|940951| 1365|
|972032|854531| 1276|
|972032|879259| 1253|
|969330|936124|  910|
|977370|927170|  910|
|983514|930358|  870|
+------+------+-----+
only showing top 20 rows



In [15]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, sum

window = Window.partitionBy("artist").orderBy(col("count").desc())
       
topsTracksPerArtist = tracksPerArtist.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= 100) \
        .drop(col("row_number")) \
        .cache()

In [17]:
sumsTracksPerArtist = topsTracksPerArtist.groupBy(col("artist")) \
.agg(sum(col("count")).alias("sum_weights")) \
.orderBy("artist") \
.cache()

In [18]:
normalized_topsTracksPerArtist = topsTracksPerArtist.join(sumsTracksPerArtist, "artist", "inner") \
    .withColumn("norm_weight", col("count") / col("sum_weights")) \
    .cache()

In [21]:
results = normalized_topsTracksPerArtist.orderBy(col("norm_weight").desc(), col("artist"), col("track")) \
    .limit(40) \
    .select(col("artist"),col("track"))

In [22]:
for u, t in results.collect():
    print("{} {}".format(u, t))

967993 869415
967998 947428
968004 927380
968017 859321
968022 852786
968034 807671
968038 964150
968042 835935
968043 913568
968046 935077
968047 806127
968065 907906
968073 964586
968086 813446
968092 837129
968118 914441
968125 821410
968140 953008
968148 877445
968161 809793
968163 803065
968168 876119
968189 858639
968221 896937
968224 892880
968232 825536
968237 932845
968238 939177
968241 879045
968242 911250
968248 953554
968255 808494
968259 880230
968265 950148
968266 824437
968269 913243
968272 816049
968278 946743
968285 847460
968286 940006
